In [1]:
import pandas as pd
from tg.grammar_ru.common import Loc
from tg.grammar_ru.ml.corpus import CorpusReader
import re
import numpy as np
import zipfile
import io

# Loading data

In [2]:
reader = CorpusReader(Loc.corpus_path/'lenta.base.zip')

In [3]:
frames = reader.get_frames()

In [5]:
df = frames.first()

In [9]:
df.head(20)

,word_id,sentence_id,word_index,paragraph_id,word_tail,word,word_type,word_length,file_id,corpus_id
0,0,0,0,0,1,Американские,ru,12,bbd53cdb-f9c2-4ee9-8a0e-0238e52253d8,lenta.base.zip
1,1,0,1,0,1,войска,ru,6,bbd53cdb-f9c2-4ee9-8a0e-0238e52253d8,lenta.base.zip
2,2,0,2,0,1,продвигаются,ru,12,bbd53cdb-f9c2-4ee9-8a0e-0238e52253d8,lenta.base.zip
3,3,0,3,0,1,к,ru,1,bbd53cdb-f9c2-4ee9-8a0e-0238e52253d8,lenta.base.zip
4,4,0,4,0,1,центру,ru,6,bbd53cdb-f9c2-4ee9-8a0e-0238e52253d8,lenta.base.zip
5,5,0,5,0,1,города,ru,6,bbd53cdb-f9c2-4ee9-8a0e-0238e52253d8,lenta.base.zip
6,6,0,6,0,0,Неджеф,ru,6,bbd53cdb-f9c2-4ee9-8a0e-0238e52253d8,lenta.base.zip
7,7,0,7,0,1,",",punct,1,bbd53cdb-f9c2-4ee9-8a0e-0238e52253d8,lenta.base.zip
8,8,0,8,0,1,сообщает,ru,8,bbd53cdb-f9c2-4ee9-8a0e-0238e52253d8,lenta.base.zip
9,9,0,9,0,1,агентство,ru,9,bbd53cdb-f9c2-4ee9-8a0e-0238e52253d8,lenta.base.zip


# Чтобы что бы

In [6]:
def get_words(df):
    what = df[df['word'] == 'что']
    what_next = what[['sentence_id', 'word_index']].copy()
    what_next['word_index'] += 1
    what_neighbour = df.merge(what_next, on=['sentence_id', 'word_index'], how='inner')
    
    return what_neighbour[what_neighbour['word'] == 'бы']

In [7]:
would = get_words(df)
would

,word_id,sentence_id,word_index,paragraph_id,word_tail,word,word_type,word_length,original_word_id,original_sentence_id,original_paragraph_id,updated,file_id,corpus_id
438,241296,197321,23,195180,1,бы,ru,2,46296,2321,180,False,092b10a4-1328-4cd6-bae6-0367fd502a19,lenta.base.zip


In [154]:
what_with_pair_loc = would[['sentence_id', 'word_index', 'word_id']].copy()
what_with_pair_loc['word_index'] -= 1
what_with_pair_loc

,sentence_id,word_index,word_id
1402,1716034,9,1858680


In [147]:
what_with_pair_loc.index

Int64Index([1402], dtype='int64')

In [106]:
df.merge(what_with_pair_loc, on=['sentence_id', 'word_index'], how='inner')

,word_id,sentence_id,word_index,paragraph_id,word_tail,word,word_type,word_length,original_word_id,original_sentence_id,original_paragraph_id,updated,file_id,corpus_id
0,1858679,1716034,9,1708643,1,что,ru,3,150684,8039,648,False,0623305a-1d75-4e32-ae48-763b2f7aa81c,lenta.base.zip


In [163]:
df.loc[what_with_pair_loc['word_id'] - 1, 'word'] = 'что'

In [177]:
what_with_pair_loc['word_id'].values

array([1858680])

In [169]:
df[df['word_id'] == 1858679]

,word_id,sentence_id,word_index,paragraph_id,word_tail,word,word_type,word_length,original_word_id,original_sentence_id,original_paragraph_id,updated,file_id,corpus_id
1858679,1858679,1716034,9,1708643,1,что,ru,3,150684,8039,648,False,0623305a-1d75-4e32-ae48-763b2f7aa81c,lenta.base.zip


In [178]:
df.drop(what_with_pair_loc['word_id'])

,word_id,sentence_id,word_index,paragraph_id,word_tail,word,word_type,word_length,original_word_id,original_sentence_id,original_paragraph_id,updated,file_id,corpus_id
1707995,1707995,1707995,0,1707995,1,Комитет,ru,7,0,0,0,False,0623305a-1d75-4e32-ae48-763b2f7aa81c,lenta.base.zip
1707996,1707996,1707995,1,1707995,1,по,ru,2,1,0,0,False,0623305a-1d75-4e32-ae48-763b2f7aa81c,lenta.base.zip
1707997,1707997,1707995,2,1707995,1,этике,ru,5,2,0,0,False,0623305a-1d75-4e32-ae48-763b2f7aa81c,lenta.base.zip
1707998,1707998,1707995,3,1707995,1,Международной,ru,13,3,0,0,False,0623305a-1d75-4e32-ae48-763b2f7aa81c,lenta.base.zip
1707999,1707999,1707995,4,1707995,1,федерации,ru,9,4,0,0,False,0623305a-1d75-4e32-ae48-763b2f7aa81c,lenta.base.zip
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1933179,1933179,1720099,30,1708988,1,до,ru,2,225184,12104,993,False,0623305a-1d75-4e32-ae48-763b2f7aa81c,lenta.base.zip
1933180,1933180,1720099,31,1708988,1,750,unk,3,225185,12104,993,False,0623305a-1d75-4e32-ae48-763b2f7aa81c,lenta.base.zip
1933181,1933181,1720099,32,1708988,0,миллиардов,ru,10,225186,12104,993,False,0623305a-1d75-4e32-ae48-763b2f7aa81c,lenta.base.zip
1933182,1933182,1720099,33,1708988,0,),punct,1,225187,12104,993,False,0623305a-1d75-4e32-ae48-763b2f7aa81c,lenta.base.zip


In [104]:
what_with_pair_index = df.merge(what_with_pair_loc, on=['sentence_id', 'word_index'], how='inner').index
what_with_pair_index

Int64Index([0], dtype='int64')

In [101]:
df['sentence_id']

1707995     0
1707996     1
1707997     2
1707998     3
1707999     4
           ..
1933179    30
1933180    31
1933181    32
1933182    33
1933183    34
Name: word_index, Length: 225189, dtype: int64

# Training

In [1]:
from pathlib import Path

from tg.grammar_ru.common import Loc
from research.common import run_training


bundle_name = 'big'
bundle_path = Loc.bundles_path/f'chtoby/{bundle_name}'

In [2]:
result = run_training.run_local(bundle_path)

2022-11-23 09:55:20.780737+00:00 INFO: Training starts. Info: {'name': 'TSAG-'}
2022-11-23 09:55:20.871025+00:00 INFO: Ensuring/loading bundle. Bundle before:
{'slovnet': {'shape': (6747861, 18), 'index_name': 'word_id'}, 'syntax_fixes': {'shape': (6747861, 4), 'index_name': 'word_id'}, 'syntax_stats': {'shape': (6747861, 6), 'index_name': 'word_id'}, 'index': {'shape': (266027, 5), 'index_name': 'sample_id'}, 'syntax_closure': {'shape': (18338761, 4), 'index_name': 'entry_id'}, 'src': {'shape': (6747861, 18), 'index_name': None}, 'pymorphy': {'shape': (6747861, 16), 'index_name': 'word_id'}}
2022-11-23 09:55:20.899514+00:00 INFO: Bundle loaded
{'slovnet': {'shape': (6747861, 18), 'index_name': 'word_id', 'columns': ['POS', 'Case', 'Degree', 'Gender', 'Number', '...'], 'index': [0, 1, 2, 3, 4, '...']}, 'syntax_fixes': {'shape': (6747861, 4), 'index_name': 'word_id', 'columns': ['syntax_parent_id', 'root', 'cycle_status', 'correct_root'], 'index': [0, 1, 2, 3, 4, '...']}, 'syntax_stats'

KeyboardInterrupt: 